In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('indonesian')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.corpus.stopwords.words('indonesian')
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Activation, Dropout, SpatialDropout1D, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
import numpy as np
import pandas as pd
from sklearn.model_selection import  train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import  accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading indonesian: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
df = pd.read_csv("train_preprocess.tsv.txt", sep="\t", engine="python", names=["data", "label"])


In [3]:
df

,data,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [4]:
df.shape

(11000, 2)

In [5]:
def cleansing(text):
    # Make sentence being lowercase
    text = text.lower()

    # Remove user, rt, \n, retweet, \t, url, xd
    pattern_1 = r'(user|retweet|\\t|\\r|url|xd)'
    text = re.sub(pattern_1, '', text)

    # Remove mention
    pattern_2 = r'@[^\s]+'
    text = re.sub(pattern_2, '', text)

    # Remove hashtag
    pattern_3 = r'#([^\s]+)'
    text = re.sub(pattern_3, '', text)

    # Remove general punctuation, math operation char, etc.
    pattern_4 = r'[\,\@\*\_\-\!\:\;\?\'\.\"\)\(\{\}\<\>\+\%\$\^\#\/\`\~\|\&\|]'
    text = re.sub(pattern_4, ' ', text)

    # Remove single character
    pattern_5 = r'\b\w{1,3}\b'
    text = re.sub(pattern_5, '', text)

    # Remove emoji
    pattern_6 = r'\\[a-z0-9]{1,5}'
    text = re.sub(pattern_6, '', text)

    # Remove digit character
    pattern_7 = r'\d+'
    text = re.sub(pattern_7, '', text)

    # Remove url start with http or https
    pattern_8 = r'(https|https:)'
    text = re.sub(pattern_8, '', text)

    # Remove (\); ([); (])
    pattern_9 = r'[\\\]\[]'
    text = re.sub(pattern_9, '', text)

    # Remove character non ASCII
    pattern_10 = r'[^\x00-\x7f]'
    text = re.sub(pattern_10, '', text)

    # Remove character non ASCII
    pattern_11 = r'(\\u[0-9A-Fa-f]+)'
    text = re.sub(pattern_11, '', text)

    # Remove multiple whitespace
    pattern_12 = r'(\s+|\\n)'
    text = re.sub(pattern_12, ' ', text)
    
    # Remove whitespace at the first and end sentences
    text = text.rstrip()
    text = text.lstrip()
    return text

indo_stop_words = stopwords.words("indonesian")

def remove_stopwords(text):
    return ' '.join([word for word in word_tokenize(text) if word not in indo_stop_words])

def tokenisasi(text):
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens

def stemming(tokens):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return ' '.join([stemmer.stem(token) for token in tokens])

In [6]:
df_coba = df.applymap(cleansing)
#df_coba = df_coba.applymap(remove_stopwords)
#df_coba = df_coba.applymap(stemming)
df_coba['word_token'] = df_coba['data'].apply(tokenisasi)


In [7]:
# Menggabungkan setiap elemen dalam kolom 'word_tokens' menjadi satu string
df_coba['string_token'] = df_coba['word_token'].apply(' '.join)

In [8]:
# membuat dictionary mapping
#mapping = {'positive': 1, 'neutral': 0, 'negative': 2}

In [9]:
# mengubah nilai pada kolom 'label' dengan method apply
#df_coba['label'] = df_coba['label'].apply(lambda x: [mapping[i] for i in x]) --> hanya jika isi kolom 'label' berbentuk list atau array
#df_coba['label'] = df_coba['label'].map(mapping)

In [10]:
le = LabelEncoder()

df_coba['label_encode'] = le.fit_transform(df_coba['label'])

display(df_coba.head())

print(df_coba.dtypes)

,data,label,word_token,string_token,label_encode
0,warung dimiliki oleh pengusaha pabrik tahu yan...,positive,"[warung, dimiliki, oleh, pengusaha, pabrik, ta...",warung dimiliki oleh pengusaha pabrik tahu yan...,2
1,mohon ulama lurus k mmbri hujjah partai yang h...,neutral,"[mohon, ulama, lurus, k, mmbri, hujjah, partai...",mohon ulama lurus k mmbri hujjah partai yang h...,1
2,lokasi strategis jalan sumatera bandung tempat...,positive,"[lokasi, strategis, jalan, sumatera, bandung, ...",lokasi strategis jalan sumatera bandung tempat...,2
3,betapa bahagia diri saat unboxing paket barang...,positive,"[betapa, bahagia, diri, saat, unboxing, paket,...",betapa bahagia diri saat unboxing paket barang...,2
4,jadi mahasiswa jangan sombong dong kasih kartu...,negative,"[jadi, mahasiswa, jangan, sombong, dong, kasih...",jadi mahasiswa jangan sombong dong kasih kartu...,0


data            object
label           object
word_token      object
string_token    object
label_encode     int32
dtype: object


In [11]:
df_coba.isna().sum()

data            0
label           0
word_token      0
string_token    0
label_encode    0
dtype: int64

# CNN

In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
#data = pd.read_csv("train_preprocess.tsv.txt", sep="\t", engine="python", names=["data", "label"])
sentences = df_coba['word_token'].to_list()
labels = df_coba['label_encode'].to_list()



# Convert labels to numeric values
#label_dict = {"negative": 0, "neutral": 1, "positive": 2}
#labels = np.array([label_dict[label] for label in labels])
#labels = np.array([labels])

# Tokenize data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=250)

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(padded_sequences, labels, test_size=0.2)

# One-hot encode the labels
Y_train = tf.keras.utils.to_categorical(Y_train, 3)
Y_test = tf.keras.utils.to_categorical(Y_test, 3)

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=250),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 2
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test))

# Evaluate the model on test set
test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=2)
print("Test accuracy:", test_acc)


Epoch 1/2
275/275 [==============================] - 43s 144ms/step - loss: 0.5730 - accuracy: 0.7561 - val_loss: 0.4144 - val_accuracy: 0.8368
Epoch 2/2
275/275 [==============================] - 40s 144ms/step - loss: 0.3026 - accuracy: 0.8883 - val_loss: 0.3766 - val_accuracy: 0.8623
69/69 - 1s - loss: 0.3766 - accuracy: 0.8623 - 1s/epoch - 17ms/step
Test accuracy: 0.8622727394104004


In [14]:
# mengubah nilai pada kolom 'label' menjadi integer
#df_coba['label'] = df_coba['label'].apply(lambda x: x[0]).astype(int)

In [15]:
#text = df_coba['word_token'].tolist()
#print(len(text))

In [16]:
#token = Tokenizer()
#token.fit_on_texts(text)

In [17]:
#df_coba['word_to_index'] = token.texts_to_sequences(text)

# Save CNN Model

In [18]:
model.save("sentiment_analysis_model_CNN_challenge.h5")


## preprocessing tambahan untuk LSTM

In [36]:
df_copy = df_coba.copy()
df_copy.drop(columns=["data","label"], inplace=True)

In [37]:
df_copy

,word_token,string_token,label_encode
0,"[warung, dimiliki, oleh, pengusaha, pabrik, ta...",warung dimiliki oleh pengusaha pabrik tahu yan...,2
1,"[mohon, ulama, lurus, k, mmbri, hujjah, partai...",mohon ulama lurus k mmbri hujjah partai yang h...,1
2,"[lokasi, strategis, jalan, sumatera, bandung, ...",lokasi strategis jalan sumatera bandung tempat...,2
3,"[betapa, bahagia, diri, saat, unboxing, paket,...",betapa bahagia diri saat unboxing paket barang...,2
4,"[jadi, mahasiswa, jangan, sombong, dong, kasih...",jadi mahasiswa jangan sombong dong kasih kartu...,0
...,...,...,...
10995,"[tidak, kecewa]",tidak kecewa,2
10996,"[enak, rasa, masakan, apalagi, kepiting, yang,...",enak rasa masakan apalagi kepiting yang menyen...,2
10997,"[hormati, partai, partai, yang, telah, berkoal...",hormati partai partai yang telah berkoalisi,1
10998,"[pagi, pagi, pasteur, sudah, macet, parah, bik...",pagi pagi pasteur sudah macet parah bikin jeng...,0


# NAIVE BAYES

In [38]:
vectorizer = CountVectorizer()

X_train, X_test, Y_train, Y_test = train_test_split(df_copy['string_token'], df_copy['label_encode'], train_size=0.8, random_state=42)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [30]:
nb = MultinomialNB() # Panggil MODEL NB
nb.fit(X_train, Y_train) # Training data
Y_pred = nb.predict(X_test) # Prediksi data X_test


In [31]:
confusion_matrix(Y_pred, Y_test)

array([[ 528,   69,  116],
       [   5,  135,    7],
       [ 147,   35, 1158]], dtype=int64)

In [32]:
print(classification_report(Y_pred, Y_test))

              precision    recall  f1-score   support

           0       0.78      0.74      0.76       713
           1       0.56      0.92      0.70       147
           2       0.90      0.86      0.88      1340

    accuracy                           0.83      2200
   macro avg       0.75      0.84      0.78      2200
weighted avg       0.84      0.83      0.83      2200



# LSTM

In [40]:
df_copy['word_token'] = df_copy['word_token'].to_list()
df_copy['label_encode'] = df_copy['label_encode'].to_list()


In [41]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+.,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_copy['word_token'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(df_copy['word_token'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Found 15763 unique tokens.
Shape of data tensor: (11000, 250)


In [42]:
Y = pd.get_dummies(df_copy['label_encode']).values
print('Shape of label tensor:', Y.shape)


Shape of label tensor: (11000, 3)


In [43]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.10, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(9900, 250) (9900, 3)
(1100, 250) (1100, 3)


In [44]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 2
batch_size = 64

model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/2
140/140 [==============================] - 180s 1s/step - loss: 0.6075 - accuracy: 0.7536 - val_loss: 0.4370 - val_accuracy: 0.8192
Epoch 2/2
140/140 [==============================] - 173s 1s/step - loss: 0.2898 - accuracy: 0.8984 - val_loss: 0.3519 - val_accuracy: 0.8717


In [45]:
Y_pred = model.predict(X_test)
Y_pred.shape

35/35 [==============================] - 7s 122ms/step


(1100, 3)

In [46]:
#from mlxtend.preprocessing import one_hote
#result = np.where(Y_pred[0]==np.amax(Y_pred[0]))
#one_hot(result[0])

Y_pred_labels = np.argmax(Y_pred, axis=1)
print(Y_pred_labels)

[0 1 2 ... 2 2 0]


In [47]:
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)
accuracy = accuracy_score(Y_test_classes, Y_pred_classes)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.00%


In [49]:
from sklearn.metrics import classification_report

Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_test_classes = np.argmax(Y_test, axis=1)
print(classification_report(Y_test_classes, Y_pred_classes))

              precision    recall  f1-score   support

           0       0.81      0.88      0.84       342
           1       0.85      0.78      0.81       116
           2       0.93      0.90      0.91       642

    accuracy                           0.88      1100
   macro avg       0.86      0.85      0.86      1100
weighted avg       0.88      0.88      0.88      1100



# Save LSTM Model

In [50]:
model.save("sentiment_analysis_model_LSTM_challenge.h5")

# Cara Menggunakan model kembali dengan load model

In [ ]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import load_model

# load data baru
df_new = pd.read_csv("train_preprocess.tsv.txt", sep="\t", engine="python", names=["data", "label"])
df_new = df_new.applymap(cleansing)
df_new = df_new.applymap(remove_stopwords)

# drop kolom label
df_new.drop(columns=['label'], inplace=True)

# lakukan preprocessing pada data baru
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df_new['data'].values)
X_new = tokenizer.texts_to_sequences(df_new['data'].values)
X_new = pad_sequences(X_new, maxlen=MAX_SEQUENCE_LENGTH)

loaded_model = load_model("D:\github branch davinca\challenge_platinum_binar_academy\sentiment_analysis_model_challenge.h5")

# lakukan prediksi pada data baru
Y_prob = loaded_model.predict(X_new)
Y_pred = Y_prob.argmax(axis=-1)

# konversi nilai prediksi menjadi label sentimen
labels = {0: "negative", 1: "neutral", 2: "positive"}
df_new["label_sentimen"] = [labels[pred] for pred in Y_pred]

344/344 [==============================] - 34s 97ms/step


: 

In [ ]:
df_new


,data,label_sentimen
0,warung dimiliki pengusaha pabrik puluhan terke...,positive
1,mohon ulama lurus k mmbri hujjah partai diwlh ...,positive
2,lokasi strategis jalan sumatera bandung nyaman...,positive
3,betapa bahagia unboxing paket barang bagus men...,positive
4,mahasiswa sombong kasih kartu kuning belajar u...,negative
...,...,...
10995,kecewa,positive
10996,enak masakan kepiting menyenangkan memilih kep...,positive
10997,hormati partai partai berkoalisi,positive
10998,pagi pagi pasteur macet parah bikin jengkel,negative


: 

## Cara menggunakan kembali model yang telah dibuat (load model)

In [ ]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import load_model

# load data baru
df_new = pd.read_csv("original_data.csv", encoding='latin-1')
df_new['Tweet'] = df_new['Tweet'].apply(cleansing)
df_new['Tweet'] = df_new['Tweet'].apply(remove_stopwords)

# lakukan preprocessing pada data baru
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df_new['Tweet'].values)
X_new = tokenizer.texts_to_sequences(df_new['Tweet'].values)
X_new = pad_sequences(X_new, maxlen=MAX_SEQUENCE_LENGTH)

loaded_model = load_model("D:\github branch davinca\challenge_platinum_binar_academy\sentiment_analysis_model_challenge.h5")

# lakukan prediksi pada data baru
Y_prob = loaded_model.predict(X_new)
Y_pred = Y_prob.argmax(axis=-1)

# konversi nilai prediksi menjadi label sentimen
labels = {0: "negative", 1: "neutral", 2: "positive"}
df_new["label"] = [labels[pred] for pred in Y_pred]

412/412 [==============================] - 43s 103ms/step


: 

In [ ]:
df_new

,Unnamed: 0,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,label
0,0,disaat cowok berusaha melacak perhatian lantas...,1,1,1,0,0,0,0,0,1,1,0,0,positive
1,1,telat ngasih edan sarap bergaul cigax jifla ca...,0,1,0,0,0,0,0,0,0,0,0,0,positive
2,2,kadang berfikir percaya tuhan jatuh berkali ka...,0,0,0,0,0,0,0,0,0,0,0,0,positive
3,3,matamu sipit diliat,0,0,0,0,0,0,0,0,0,0,0,0,negative
4,4,kaum cebong kapir udah keliatan dongoknya dong...,1,1,0,1,1,0,0,0,0,0,1,0,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13164,13164,ngomong ndasmu congor sekate anjyng,1,1,1,0,0,0,1,0,0,1,0,0,neutral
13165,13165,kasur enak kunyuk,0,1,0,0,0,0,0,0,0,0,0,0,neutral
13166,13166,hati hati bisu bosan huft,0,0,0,0,0,0,0,0,0,0,0,0,positive
13167,13167,real mudah terdeteksi terkubur dahsyat ledakan...,0,0,0,0,0,0,0,0,0,0,0,0,positive


: 

In [ ]:
df_tweet_predict = pd.DataFrame({"Tweet": df_new['Tweet'],
                           "Label": df_new['label']
                        })
df_tweet_predict.head()

,Tweet,Label
0,disaat cowok berusaha melacak perhatian lantas...,positive
1,telat ngasih edan sarap bergaul cigax jifla ca...,positive
2,kadang berfikir percaya tuhan jatuh berkali ka...,positive
3,matamu sipit diliat,negative
4,kaum cebong kapir udah keliatan dongoknya dong...,positive


: 

In [ ]:
df_tweet_predict


,Tweet,Label
0,disaat cowok berusaha melacak perhatian lantas...,positive
1,telat ngasih edan sarap bergaul cigax jifla ca...,positive
2,kadang berfikir percaya tuhan jatuh berkali ka...,positive
3,matamu sipit diliat,negative
4,kaum cebong kapir udah keliatan dongoknya dong...,positive
...,...,...
13164,ngomong ndasmu congor sekate anjyng,neutral
13165,kasur enak kunyuk,neutral
13166,hati hati bisu bosan huft,positive
13167,real mudah terdeteksi terkubur dahsyat ledakan...,positive


: 

In [ ]:
df_tweet_predict['Label'].unique()

array(['positive', 'negative', 'neutral'], dtype=object)

: 

# --batas akhir modelling CNN dan LSTM--

# kode-kode dibawah ini sudah tidak dipakai

In [ ]:
#x = df_copy.iloc[:,1] # parameter (features)
#y = df_copy.iloc[:,0] # targeted

: 

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.7)


: 

In [ ]:
#print(x_train.shape)  # Output: (n_data_points, n_features)
#print(y_train.shape)  # Output: (n_data_points,)


: 

In [ ]:
#x_train_dtype = np.array(x_train).dtype
#print(x_train_dtype)

: 

In [ ]:
#y_train_dtype = np.array(y_train).dtype
#print(y_train_dtype)

: 

In [ ]:
#MAX_SEQUENCE_LENGTH = 100

# Lakukan padding pada setiap data input
#x_train = pad_sequences(x_train, maxlen=MAX_SEQUENCE_LENGTH)


: 

In [ ]:
#MAX_SEQUENCE_LENGTH = 100

# Lakukan padding pada setiap data input
#x_test = pad_sequences(x_test, maxlen=MAX_SEQUENCE_LENGTH)


: 

In [ ]:
#x_train = np.array(x_train, dtype=np.int64)


: 

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 25, 10), random_state=1, max_iter=500)

clf.fit(X_train, Y_train)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(50, 25, 10), max_iter=500,
              random_state=1, solver='lbfgs')

: 

In [ ]:
#y_pred = clf.predict(x_test)
#y_pred # sbetulnya masih termasuk error ini 

: 

In [ ]:
accuracy_score(Y_test, Y_pred)

0.8145454545454546

: 